In [ ]:
!pip install gremlinpython

In [ ]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *
from gremlin_python.process.traversal import Cardinality

import os

port = 8182
server = 'graphdb-develop-neptune-cluster.cluster-causxyo47g34.ap-northeast-1.neptune.amazonaws.com'
    
endpoint = f'wss://{server}:{port}/gremlin'

graph=Graph()

connection = None

try:
    connection = DriverRemoteConnection(endpoint, 'g',
                                        transport_factory=lambda: AiohttpTransport(call_from_event_loop=True))

    g = graph.traversal().withRemote(connection)

    g.V().drop().iterate()
    g.addV('person').property(T.id, '10').property('name', 'satoshi').iterate()

#     all_nodes = g.V().valueMap(True).toList()
#     print("All Nodes:", all_nodes)

    node = g.V('10').valueMap(True).toList()
    print("Node:", node)
    
    g.V('10').property('address', 'japan').iterate()
    node = g.V('10').valueMap(True).toList()
    print("Node:", node)

    g.V('10').property('address', 'hiroshima').iterate()
    node = g.V('10').valueMap(True).toList()
    print("Node:", node)

    g.V('10').property(Cardinality.single, 'name', 'satoshi256kbyte').iterate()
    node = g.V('10').valueMap(True).toList()
    print("Node:", node)
    
finally:
    if connection is not None:
        connection.close()